In [21]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import joblib

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [11]:
#data preprocessing. 

from nltk.corpus import stopwords
import nltk

# Download common resources
nltk.download("stopwords")     # for stopword removal
nltk.download("punkt")         # for tokenization
nltk.download("wordnet")       # for lemmatization
nltk.download("omw-1.4")       # WordNet lemmatizer support
nltk.download("averaged_perceptron_tagger")  # for POS tagging
nltk.download("maxent_ne_chunker")           # for named entity recognition
nltk.download("words")                       # for NER support
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ljm47\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ljm47\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ljm47\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ljm47\AppData\Roaming\nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ljm47\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\ljm47\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\ljm47\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.z

In [ ]:
a = "my name is leon"
text = a.split()   #the string is converted into a list where each element is one word in the list and we can loop over the list now.
print(text)
b = " ".join(text)
print(b)

['my', 'name', 'is', 'leon']
my name is leon


In [12]:

df = normalize_text(df)
df.head()


,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [13]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [14]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]
df.head(3)

,sentiment,content
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
6,sadness,sleep im not thinking old friend want married ...


In [15]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()

C:\Users\ljm47\AppData\Local\Temp\ipykernel_11744\1089524538.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
C:\Users\ljm47\AppData\Local\Temp\ipykernel_11744\1089524538.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [16]:
vectorizer = CountVectorizer(max_features=1000)  # BOW with 1000 most common words
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [17]:
X

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 55969 stored elements and shape (10374, 1000)>

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
import dagshub
#Now from next cell onwards we will log the parameters of BOW and model 

mlflow.set_tracking_uri('https://dagshub.com/leonmoraes17/mlops-mini-project.mlflow')
dagshub.init(repo_owner='leonmoraes17', repo_name='mlops-mini-project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

Accessing as leonmoraes17

Initialized MLflow to track repo "leonmoraes17/mlops-mini-project"

Repository leonmoraes17/mlops-mini-project initialized!

2025/08/30 19:06:44 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/7da7e6a3b79246c99a5ffed50a17734e', creation_time=1756595204794, experiment_id='0', last_update_time=1756595204794, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [22]:
with mlflow.start_run():
    
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    # mlflow.sklearn.log_model(model, "model")

    # Save and log the notebook  . if this .py file then mlflow.log_artifact(__file__)
    import os
    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

Accuracy: 0.7749397590361445
Precision: 0.7660194174757281
Recall: 0.7773399014778325
F1 Score: 0.7716381418092909
🏃 View run gifted-eel-752 at: https://dagshub.com/leonmoraes17/mlops-mini-project.mlflow/#/experiments/0/runs/de4174e364654d09b34075e852d2b60f
🧪 View experiment at: https://dagshub.com/leonmoraes17/mlops-mini-project.mlflow/#/experiments/0


In [23]:
joblib.dump(model, "D:\Projects\mlops_miniproject\mlops-mini-project\src\model\model.pkl")

['D:\\Projects\\mlops_miniproject\\mlops-mini-project\\src\\model\\model.pkl']

In [24]:
mlflow.log_artifact("D:\Projects\mlops_miniproject\mlops-mini-project\src\model\model.pkl", artifact_path="model")

In [ ]:
# hyperparameter tuning

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os
import dagshub



# Load the data
# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])


# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

# Normalize the text data
df = normalize_text(df)

x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Define hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}



    # Perform grid search
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)

    # Log each parameter combination as a child run
for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
 
    #{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'} 1st iterations of grid_search.cv_results_{'params'}
    # **params will unpack the dictionary, so model = LR(c=0.1, penalty= l1, solver = liblinear)
    model = LogisticRegression(**params)  #
    model.fit(X_train, y_train)
            
            # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(params) 
    print(accuracy)       
    print(grid_search.cv_results_['params'])
            
            
            

C:\Users\ljm47\AppData\Local\Temp\ipykernel_11744\3279466112.py:83: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}
0.7397590361445783
[{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}]
{'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
0.7893975903614457
[{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}]
{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
0.7826506024096386
[{'C': 0.1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}, {'C': 1, 'penalty': 'l2